In [1]:
import pickle
import os
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import FastICA
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import nibabel as nib
import ants
from scipy.ndimage.morphology import binary_erosion
from scipy.ndimage.morphology import binary_dilation
import cv2
import bigbadbrain as bbb
import scipy
from scipy.interpolate import interp1d
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import cm
import tqdm
import ffmpeg
import brainsss

/home/users/brezovec/.local/lib/python3.6/site-packages/ants/viz/render_surface_function.py:16: UserWarning: Cant import Plotly. Install it `pip install chart_studio` if you want to use ants.render_surface_function
  "Cant import Plotly. Install it `pip install chart_studio` if you want to use ants.render_surface_function"


In [2]:
cluster_dir = "/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20221109_cluster_pca/"

In [3]:
file = os.path.join(cluster_dir, 'superclusters_more.npy')
superclusters = np.load(file)
#with 249 there will be clusters 1-250 inclusive (len=250). cluster 140 is background
superclusters_250 = superclusters[:,:,:,249]
# right brain gets +250 to cluster ID so now 500 clusters
superclusters_250[157:,:,:] += 250

In [4]:
superclusters_250.shape

(314, 146, 91)

In [5]:
atlas_clean = brainsss.load_roi_atlas()
explosion_rois = brainsss.load_explosion_groups()
all_rois = brainsss.unnest_roi_groups(explosion_rois)
roi_masks = brainsss.make_single_roi_masks(all_rois, atlas_clean)
roi_contours = brainsss.make_single_roi_contours(roi_masks, atlas_clean)

In [11]:
atlas_IDs = brainsss.load_roi_hemi_ids()

In [27]:
cluster_rois = {}
for cluster in range(251):
    cluster_rois[cluster] = {}
    ind = np.where(superclusters_250==cluster)
    rois_in_SC = atlas_clean[ind]
    total = 0
    for roi in np.unique(rois_in_SC):
        count = np.sum(rois_in_SC==roi)
        try:
            cluster_rois[cluster][atlas_IDs[0]['left'][int(roi)]] = count
            total += count
        except:
            pass
    for roi in np.unique(rois_in_SC):
        try:
            cluster_rois[cluster][atlas_IDs[0]['left'][int(roi)]] /= total
        except:
            pass

In [69]:
rois = list(atlas_IDs[0]['left'].values())

SC_in_rois_sorted_all = {}
for roi in rois:
    SC_in_rois = []
    for cluster in range(251):
        try:
            percent = cluster_rois[cluster][roi]
            SC_in_rois.append((cluster,percent))
        except:
            pass

    sort_ind = list(np.argsort(np.asarray(SC_in_rois)[:,1]))[::-1]
    SC_in_rois_sorted = [SC_in_rois[i] for i in sort_ind]
    SC_in_rois_sorted_all[roi] = SC_in_rois_sorted

In [71]:
save_file = os.path.join(cluster_dir,"20221203_SC_atlas_ROIs")
np.save(save_file,SC_in_rois_sorted_all,allow_pickle=True)

In [95]:
SC_in_rois_sorted_all['SAD']

[(76, 0.7737373737373737),
 (225, 0.7345132743362832),
 (101, 0.45213379469434833),
 (95, 0.4025423728813559),
 (128, 0.3645320197044335),
 (51, 0.29640427599611274),
 (141, 0.26153846153846155),
 (110, 0.25882352941176473),
 (19, 0.19263157894736843),
 (105, 0.18009478672985782),
 (80, 0.10277777777777777),
 (119, 0.09513023782559456),
 (155, 0.07309486780715396),
 (247, 0.061224489795918366),
 (227, 0.04395604395604396),
 (77, 0.03531073446327684),
 (230, 0.034141958670260555),
 (129, 0.016647127784290738),
 (134, 0.01168063762539508),
 (194, 0.006892357591729171),
 (140, 0.005711273174589228),
 (63, 0.004291845493562232),
 (118, 0.004067520846044336),
 (39, 0.003863298662704309),
 (205, 0.0030211480362537764),
 (245, 0.0018450184501845018),
 (65, 0.0012165450121654502),
 (88, 0.0006540222367560497),
 (9, 0.0005903187721369539),
 (1, 0.00037290242386575515),
 (103, 0.00011424654404204273)]

In [85]:
rois

['PB',
 'FB',
 'EB',
 'NO',
 'BU_R',
 'LAL_R',
 'PVLP_R',
 'VES_R',
 'EPA_R',
 'GOR_R',
 'AMMC_R',
 'AL_R',
 'MB_PED_R',
 'MB_VL_R',
 'MB_ML_R',
 'MB_CA_R',
 'AVLP_R',
 'IVLP_R',
 'CAN_R',
 'FLA_R',
 'PRW',
 'SAD',
 'GNG',
 'SLP_R',
 'SMP_R',
 'PLP_R',
 'LH_R',
 'IPS_R',
 'SPS_R',
 'CRE_R',
 'SCL_R',
 'ICL_R',
 'ATL_R',
 'IB_R',
 'SIP_R',
 'MED_R',
 'LO_R',
 'LP_R',
 'AOTU_R']